In [1]:
from download_day import download_day
from load_saved_fd4 import load_saved_fd4
from scrape_aircraft_list import scrape_aircraft_list
from get_perf_model_typecodes import get_perf_model_typecodes 
from match_icao_model import match_icao_model
import pandas as pd
from time import sleep

Saved typecodes_available to /scratch/omg28/Data/aircraftdb/performance_models_typecodes.csv


In [2]:
# User Inputs:
start_time_str       = '2024-01-01T00:00:00Z'
stop_time_str        = '2024-12-31T23:59:59Z'
query_limit          = 30e4
send_notification    = True
make_plot            = False
output_dir           = "/scratch/omg28/Data/no_track_2024"

Section 1. Download flight information from Opensky

In [ ]:
# Download data from OpenSky history database
download_day(start_time_str, stop_time_str, query_limit, send_notification, make_plot, output_dir)

In [ ]:
# load the saved day of data from a .pkl file to a pandas dataframe
loaded_day = load_saved_fd4(stop_time_str, output_dir, query_limit)

Section 2. Scrape the EUCONTROL database for aircraft flight performance information

In [ ]:
# scrape the list of aircraft with performance models from the EUCONTROL website
aircraft_list = scrape_aircraft_list()
# get the performance model type codes from the scraped aircraft list
perf_model_typecodes = get_perf_model_typecodes(aircraft_list)

Section 3. Process the aircraft performance and flight data, retaining only flights we have information about.

In [ ]:
# process the loaded time period of data.
# - matches 24 bit transponder codes from Feb 2025 aircraft metadata database with the ICAO typecode
# - removes all flights for which the transponder code does not have a corresponding ICAO typecode
# - removes all flights for which the takeoff OR landing airport is unknown
complete_flights_typecode_assigned = match_icao_model(start_time_str, stop_time_str, query_limit)


In [ ]:
complete_flights_typecode_assigned.head()

complete_flights_typecode_assigned = pd.load_pickle('/scratch/omg28/Data/no_track_2024/complete_flights_typecode_assigned.pkl')

# inner join the loaded day of data with the scraped aircraft list
# - this will remove all flights for which the ICAO typecode does not have a corresponding performance model

# first, load in the complete list of aircraft for which there is a performance model:
aircraft_with_perf_model = pd.read_pickle('/scratch/omg28/Data/aircraftdb/performance_models_typecodes.pkl')



complete_flights_perf_model = pd.merge(complete_flights_typecode_assigned, perf_model_typecodes, how='inner', left_on='icao24', right_on='icao24')